# Modelagem preditiva

A modelagem preditiva são modelos computacionais que usam a matemática para prever resultados futuros. Os algoritmos preditivos usam dados de treino para aprender sobre o comportamento das variáveis afim de buscar a aproximação da função alvo, a função alvo é a formulação matemática que explica a relação entre os dados de entrada e de saída.

A análise preditiva está sendo utilizada por empresas e indivíduos em todo o mundo para extrair valor de dados históricos, após ser desenvolvido e validado, o modelo preditivo consegue generalizar o conhecimento aprendido dos dados históricos para prever o futuro.

### Importando as bibliotecas necessárias

Para fazer nosso trabalho precisamos das funções e métodos por isso vamos importar as bibliotecas do python.

In [1]:
import pandas as pd
import numpy as np

from tpot import TPOTRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score 

C:\Users\lucas.a.ferreira\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
import warnings
warnings.filterwarnings('ignore')

### Carregando o conjunto de dados

In [4]:
# Carregando o conjunto de dados e salvando na variável 'imoveis'.
imoveis = pd.read_csv('../imoveis_base_com_outliers.csv')

# Vamos tirar as colunas 'Unnamed: 0', 'Link', 'Estado', 'Cidade' e 'Descrição'
imoveis = imoveis.drop(['Unnamed: 0', 'Link', 'Estado', 'Descrição'], axis=1)

In [5]:
pd.set_option('display.max_columns', 100)

# Verificando o dataset
imoveis.head()

,Cidade,Valor_aluguel,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda
0,belo horizonte,3250.0,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,belo horizonte,4800.0,1100.0,0.0,70.0,2.0,2.0,1.0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1
2,belo horizonte,7300.0,3300.0,1100.0,190.0,4.0,4.0,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
3,belo horizonte,3100.0,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,belo horizonte,1500.0,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1


In [7]:
# Verificando a dimensão do nosso dataset
imoveis.shape

(19623, 34)

In [8]:
df = imoveis

Nesse modelo vamos usar também a variável 'Cidade' porém como essa variável é do tipo texto temos que trata-lá para que o algoritmo possa receber ela da forma certa, em outras palavras como valor numérico. O pacote LabelEncoder transforma rótulos de texto em números.

In [9]:
labelencoder = LabelEncoder()
df['Cidade'] = labelencoder.fit_transform(df['Cidade'])

In [10]:
# Separando a variável target
X = df.drop(['Valor_aluguel'], axis=1)
y = df['Valor_aluguel'].values

In [11]:
X

,Cidade,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda
0,2,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1100.0,0.0,70.0,2.0,2.0,1.0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1
2,2,3300.0,1100.0,190.0,4.0,4.0,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
3,2,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,2,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19618,23,1500.0,0.0,344.0,4.0,4.0,6.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
19619,23,1400.0,0.0,175.0,4.0,2.0,4.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
19620,23,2300.0,0.0,243.0,4.0,2.0,5.0,0,0,1,1,0,0,0,1,1,0,1,1,1,1,0,1,1,1,0,0,1,1,1,1,0,1
19621,23,0.0,0.0,400.0,4.0,5.0,5.0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0


In [12]:
y

array([ 3250.,  4800.,  7300., ...,  9015., 25000.,  7500.])

### Criando dados de treino e teste

In [13]:
# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### Modelo com o Tpot AutoML

O objetivo do TPOT é automatizar a construção de pipelines de machine learning combinando uma representação de árvore de expressão flexível de pipelines com algoritmos de pesquisa. O TPOT usa a biblioteca scikit-learn baseada em Python.

In [14]:
model = TPOTRegressor(generations=1, 
                      population_size=100, 
                      scoring='r2', 
                      cv=2, verbosity=2, 
                      random_state=0)

In [15]:
%%time
model.fit(X_train, y_train)

[08:02:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:29] WARNI

[08:02:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:03:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:03:41] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:04:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:04:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:04:17] WARNI

[08:09:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:41] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:11:39] WARNI

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=2,
              disable_update_check=False, early_stop=None, generations=1,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=0, scoring='r2', subsample=1.0,
              template='RandomTree', use_dask=False, verbosity=2,
              warm_start=False)

### Exportando a pipeline de execução

In [16]:
model.export('pipeline.py')
with open('pipeline.py') as pipe:
    print("".join(pipe.readlines()))

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=0)

# Average CV score on the training set was:0.007252132143774737
exported_pipeline = GradientBoostingRegressor(alpha=0.99, learning_rate=0.5, loss="huber", max_depth=9, max_features=0.1, min_samples_leaf=10, min_samples_split=5, n_estimators=100, subsample=0.7500000000000001)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)



### Executando a pipeline

In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

model_tpot = GradientBoostingRegressor(alpha=0.99, learning_rate=0.5, loss="huber", max_depth=9,\
                                       max_features=0.1, min_samples_leaf=10, min_samples_split=5, n_estimators=100,\
                                       subsample=0.7500000000000001)

model_tpot.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.99, criterion='friedman_mse', init=None,
                          learning_rate=0.5, loss='huber', max_depth=9,
                          max_features=0.1, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=10, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.7500000000000001,
                          tol=0.0001, validation_fraction=0.1, verbose=0,
                          warm_start=False)

In [18]:
result = model_tpot.predict(X_test)
result

array([-448.03557857, 1681.30823047, 2032.27205231, ..., 1533.37385753,
        555.93572446, 1476.86347444])

In [19]:
R2 = r2_score(y_test,result)
MAE = mean_absolute_error(y_test,result)
print(R2)
print(MAE)

0.005871699884997916
1096.7327209161358


--------------------------------------

## Métricas de avaliação

### R-Quadrado (R²)

O R-Quadrado, ou Coeficiente de Determinação, é uma métrica que visa expressar a quantidade da variança dos dados que é explicada pelo modelo. Em outras palavras, essa medida calcula qual a porcentagem da variança que pôde ser prevista pelo modelo de regressão e, portanto, nos diz o quão “próximo” as medidas reais estão do nosso modelo.

O valor do seu R-Quadrado varia de 0 a 1 e geralmente é representado em porcentagem. Por exemplo, um R² = 75% nos diz que 75% da variância de nossos dados podem ser explicados pelo modelo construído, enquanto os outros 25%, teoricamente, se tratariam de uma variância residual.

### Erro Absoluto Médio (MAE)

O Erro Absoluto Médio consiste na média das distâncias entre valores preditos e reais. Em outras palavras, tira-se média da diferença absoluta dos valores preditos e real.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
(abs(y_test - result).sum() / len(result))

### Erro Quadrático Médio (MSE)

O Erro Quadrático Médio consiste na média do erro das previsões ao quadrado. Em outras palavras, é a média da soma dos resíduos(diferença entra os valores predito e real) ao quadrado.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
((y_test - result)**2).sum() / len(result)

-----------------------------------------------------------

### Melhor resultado do modelo:

Podemos ver pelo resultado do mean_absolute_error e do R² que não obtivemos acurácia nenhuma usando o dataset sem tratar os outliers, então podemos concluir que realmente temos que tratar os valores discrepantes.